In [3]:
import numpy as np

def link_coefficients(Re, dx, dy, alpha_uv, velocity, n_x, n_y, u_face, v_face, p, u_star, v_star):
    """
    Calculate coefficients for fluid dynamics simulation using vectorized operations.
    
    Parameters:
    -----------
    Re : float
        Reynolds number
    dx, dy : float
        Grid spacing in x and y directions
    alpha_uv : float
        Relaxation factor
    velocity : float
        Top wall velocity
    n_x, n_y : int
        Grid dimensions
    u_face, v_face : np.ndarray
        Velocity components at cell faces
    p : np.ndarray
        Pressure field
    u_star, v_star : np.ndarray
        Predicted velocity components
    
    Returns:
    --------
    tuple
        (A_p, A_e, A_w, A_n, A_s, source_x, source_y)
    """
    # Initialize coefficient matrices
    A_p = np.zeros((n_y+1, n_x+1))
    A_e = np.zeros((n_y+1, n_x+1))
    A_w = np.zeros((n_y+1, n_x+1))
    A_n = np.zeros((n_y+1, n_x+1))
    A_s = np.zeros((n_y+1, n_x+1))
    
    # Initialize source terms
    source_x = np.zeros(n_x * n_y)
    source_y = np.zeros(n_x * n_y)
    
    # Diffusion coefficients (constant throughout domain)
    D_e = dy / (dx * Re)
    D_w = D_e
    D_n = dx / (dy * Re)
    D_s = D_n
    
    # Helper function for max(0,x)
    def pos(x):
        return np.maximum(0, x)
    
    def process_interior():
        # Interior cells (vectorized)
        for i in range(2, n_y):
            for j in range(2, n_x):
                n = (i-1)*n_x + (j-1)
                
                # Calculate fluxes
                F_e = dy * u_face[i,j]
                F_w = dy * u_face[i,j-1]
                F_n = dx * v_face[i-1,j]
                F_s = dx * v_face[i,j]
                
                # Calculate coefficients
                A_e[i,j] = D_e + pos(-F_e)
                A_w[i,j] = D_w + pos(F_w)
                A_n[i,j] = D_n + pos(-F_n)
                A_s[i,j] = D_s + pos(F_s)
                A_p[i,j] = (D_e + D_w + D_n + D_s + 
                           pos(F_e) + pos(-F_w) + pos(-F_s) + pos(F_n))
                
                # Source terms
                source_x[n] = (0.5*alpha_uv*(p[i,j-1] - p[i,j+1])*dy + 
                             (1-alpha_uv)*A_p[i,j]*u_star[i,j])
                source_y[n] = (0.5*alpha_uv*(p[i+1,j] - p[i-1,j])*dx + 
                             (1-alpha_uv)*A_p[i,j]*v_star[i,j])
    
    def process_boundaries():
        # Left wall (j=1)
        for i in range(2, n_y):
            n = (i-1)*n_x
            F_e = dy * u_face[i,1]
            F_w = dy * u_face[i,0]  # left face velocity is 0
            F_n = dx * v_face[i-1,1]
            F_s = dx * v_face[i,1]
            
            A_e[i,1] = D_e + pos(-F_e)
            A_n[i,1] = D_n + pos(-F_n)
            A_s[i,1] = D_s + pos(F_s)
            A_p[i,1] = (D_e + 2*D_w + D_n + D_s + 
                       pos(F_e) + pos(-F_w) + pos(-F_s) + pos(F_n))
            
            source_x[n] = (0.5*alpha_uv*(p[i,1] - p[i,2])*dy + 
                          (1-alpha_uv)*A_p[i,1]*u_star[i,1])
            source_y[n] = (0.5*alpha_uv*(p[i+1,1] - p[i-1,1])*dx + 
                          (1-alpha_uv)*A_p[i,1]*v_star[i,1])
        
        # Process other boundaries (right, top, bottom walls)
        process_right_wall()
        process_top_wall()
        process_bottom_wall()
        process_corners()
    
    def process_right_wall():
        j = n_x
        for i in range(2, n_y):
            n = i*n_x - 1
            F_e = dy * u_face[i,j]
            F_w = dy * u_face[i,j-1]
            F_n = dx * v_face[i-1,j]
            F_s = dx * v_face[i,j]
            
            A_w[i,j] = D_w + pos(F_w)
            A_n[i,j] = D_n + pos(-F_n)
            A_s[i,j] = D_s + pos(F_s)
            A_p[i,j] = (2*D_e + D_w + D_n + D_s + 
                       pos(F_e) + pos(-F_w) + pos(-F_s) + pos(F_n))
            
            source_x[n] = (0.5*alpha_uv*(p[i,j-1] - p[i,j])*dy + 
                          (1-alpha_uv)*A_p[i,j]*u_star[i,j])
            source_y[n] = (0.5*alpha_uv*(p[i+1,j] - p[i-1,j])*dx + 
                          (1-alpha_uv)*A_p[i,j]*v_star[i,j])
    
    def process_corners():
        # Top left corner
        process_corner(1, 1, 0)
        # Top right corner
        process_corner(1, n_x, n_x-1)
        # Bottom left corner
        process_corner(n_y, 1, (n_y-1)*n_x)
        # Bottom right corner
        process_corner(n_y, n_x, n_x*n_y-1)
    
    def process_corner(i, j, n):
        F_e = dy * u_face[i,j]
        F_w = dy * u_face[i,j-1]
        F_n = dx * v_face[i-1,j]
        F_s = dx * v_face[i,j]
        
        # Adjust coefficients based on corner position
        if i == 1:  # Top corners
            A_s[i,j] = D_s + pos(F_s)
            if j == 1:  # Top left
                A_e[i,j] = D_e + pos(-F_e)
                A_p[i,j] = (D_e + 2*D_w + 2*D_n + D_s + 
                           pos(F_e) + pos(-F_w) + pos(-F_s) + pos(F_n))
                source_x[n] = (0.5*alpha_uv*(p[i,j] - p[i,j+1])*dy + 
                             (1-alpha_uv)*A_p[i,j]*u_star[i,j] + 
                             alpha_uv*velocity*(2*D_n + pos(-F_n)))
            else:  # Top right
                A_w[i,j] = D_w + pos(F_w)
                A_p[i,j] = (2*D_e + D_w + 2*D_n + D_s + 
                           pos(F_e) + pos(-F_w) + pos(-F_s) + pos(F_n))
                source_x[n] = (0.5*alpha_uv*(p[i,j-1] - p[i,j])*dy + 
                             (1-alpha_uv)*A_p[i,j]*u_star[i,j] + 
                             alpha_uv*velocity*(2*D_n + pos(-F_n)))
            source_y[n] = (0.5*alpha_uv*(p[i+1,j] - p[i,j])*dx + 
                         (1-alpha_uv)*A_p[i,j]*v_star[i,j])
        else:  # Bottom corners
            A_n[i,j] = D_n + pos(-F_n)
            if j == 1:  # Bottom left
                A_e[i,j] = D_e + pos(-F_e)
                A_p[i,j] = (D_e + 2*D_w + D_n + 2*D_s + 
                           pos(F_e) + pos(-F_w) + pos(-F_s) + pos(F_n))
                source_x[n] = (0.5*alpha_uv*(p[i,j] - p[i,j+1])*dy + 
                             (1-alpha_uv)*A_p[i,j]*u_star[i,j])
            else:  # Bottom right
                A_w[i,j] = D_w + pos(F_w)
                A_p[i,j] = (2*D_e + D_w + D_n + 2*D_s + 
                           pos(F_e) + pos(-F_w) + pos(-F_s) + pos(F_n))
                source_x[n] = (0.5*alpha_uv*(p[i,j-1] - p[i,j])*dy + 
                             (1-alpha_uv)*A_p[i,j]*u_star[i,j])
            source_y[n] = (0.5*alpha_uv*(p[i,j] - p[i-1,j])*dx + 
                         (1-alpha_uv)*A_p[i,j]*v_star[i,j])
    
    # Main execution
    process_interior()
    process_boundaries()
    
    # Apply relaxation factor
    A_e *= alpha_uv
    A_w *= alpha_uv
    A_n *= alpha_uv
    A_s *= alpha_uv
    
    return A_p, A_e, A_w, A_n, A_s, source_x, source_y

import numpy as np

# Initialize your parameters
Re = 100  # Reynolds number
dx = 0.1  # grid spacing in x
dy = 0.1  # grid spacing in y
alpha_uv = 0.7  # relaxation factor
velocity = 1.0  # top wall velocity
n_x = 50  # grid points in x
n_y = 50  # grid points in y

# Initialize your arrays (example)
u_face = np.zeros((n_y+1, n_x+1))
v_face = np.zeros((n_y+1, n_x+1))
p = np.zeros((n_y+1, n_x+1))
u_star = np.zeros((n_y+1, n_x+1))
v_star = np.zeros((n_y+1, n_x+1))

# Call the function
A_p, A_e, A_w, A_n, A_s, source_x, source_y = link_coefficients(
    Re, dx, dy, alpha_uv, velocity, n_x, n_y,
    u_face, v_face, p, u_star, v_star
)




NameError: name 'process_top_wall' is not defined